# Detection of slow-moving objects with LSST

<div>
<img src="logo_LSST.png" width="450"/>        
</div>

**Contact Author**: Antonio Vanzanella 

**Description**: This tutorial introduces the procedure of sample creation for a Dataset used to train and test a Machine learning pipeline able to detect simulated Slow-moving objects injected in the samples.  

**LSST Data Products**: Images (calexp, skyMap, MaskedImage) and Catalogs (objectTable)

**Packages**: `lsst.sphgeom`, `lsst.geom`, `lsst.afw.image`, `lsst.afw.display`, `lsst.skymap`, `lsst.daf.butler`, `scipy.interpolate`



## This Notebook
In this notebook we showed how to retrieve and collect a series of cut-outs from a sequence of images at the same sky coordinates ordered by time, <br> compressing them into archives and giving some insights on how we used them to build a Dataset and how we trained a 3D-CNN.

# Introduction

A dataset is a collection of data. Building a dataset is crucial for any machine learning application and It is the most time-consuming task. <br> Starting from the simulated images and the expected physical characteristics of the objects which we are interested in detecting, we begin to define the principal aspect that our dataset will have to need.
<br> First of all, we have to take into account the characteristics that the neural network will learn from it in its training phase. <br> We don't have much prior information about the slow-moving objects but we know that the most recognizable feature is their movement; then the problem becomes "what kind of sample could describe our object?"
<br>  The solution we found is using an LSST-simulated image representing the background and injecting into it a simulated slow-moving object. <br> To describe the object's movement, we build a kind of "video" collecting many of these images resulting in a sequence of frames.

## The whole picture

A deep Neural Network needs a lot of training samples(Training-set) to give good performances, so we put the whole procedure for creating a sample in a unique, great loop.
In this notebook, we split the code into small fragments to provide a better explanation of the single parts.

In [ ]:
import numpy
import matplotlib
import matplotlib.pyplot as plt
import pandas
pandas.set_option('display.max_rows', 1000)
from IPython.display import Markdown as md
from lsst.rsp import get_tap_service, retrieve_query
import lsst.daf.butler as dafButler
import lsst.geom
from astropy import units as u
from astropy.coordinates import SkyCoord
from random import randrange, uniform
import lsst.afw.display as afwDisplay
import lsst.afw.image as afwImage
import lsst.afw.table as afwTable
import astropy.time
from datetime import datetime
import numpy as np
from lsst.skymap import makeSkyPolygonFromBBox
lsst.afw.geom
import math
from astropy.io import fits
import lsst.sphgeom
import warnings
from astropy.wcs import WCS
from astropy.coordinates import Angle
warnings.filterwarnings('ignore')
import numpy as np
from scipy.interpolate import griddata
import glob
from PIL import Image
import re

for g in range(0,10):
    repo = 'dp02'  
    collection='2.2i/runs/DP0.2'
    butler = dafButler.Butler(repo,collections=collection)
    registry = butler.registry
    rand_RA = uniform(50,74)  #60.2837946
    rand_DEC =uniform(-44,-27) #-35.4042439
    pixelization = lsst.sphgeom.HtmPixelization(12)
    htm_id = pixelization.index(
        lsst.sphgeom.UnitVector3d(
            lsst.sphgeom.LonLat.fromDegrees(rand_RA, rand_DEC)   
        )
    )
    t = astropy.time.core.Time("2025-09-05 08:16:03.643200")
    minute = astropy.time.TimeDelta(60, format="sec")
    timespan = dafButler.Timespan(t - minute*1051200,t + minute*1051200) #4 years
    datasetRefs1 = registry.queryDatasets("calexp", htm20=htm_id,
                                             where="visit.timespan OVERLAPS my_timespan",
                                             bind={"my_timespan": timespan})
    record_query= []  
    for i, ref in enumerate(datasetRefs1):
        record_query.append(ref)
    point = lsst.geom.SpherePoint(rand_RA,rand_DEC, lsst.geom.degrees)
    cutout = []
    cutout1 = []
    cutout2 = []
    
    dataId = {'visit': record_query[0].dataId['visit'], 'detector': record_query[0].dataId['detector'], 'band': record_query[0].dataId['band']}#, 'physical_filter':(record_query[1]).dataId['physical_filter']}
    xx= butler.get('calexp', dataId=dataId)
    wwcs = xx.getWcs()
    w,h = xx.image.getDimensions()
    bbox = lsst.geom.Box2I(lsst.geom.Point2I(0, 0), lsst.geom.Extent2I(w, h))
    
    ddim= 50
    for i in range(0,len(record_query)):
        dataId = {'visit': record_query[i].dataId['visit'], 'detector': record_query[i].dataId['detector']}
        xx= butler.get('calexp', dataId= registry.expandDataId(dataId))
        radec= lsst.geom.SpherePoint(rand_RA*lsst.geom.degrees, rand_DEC*lsst.geom.degrees)
        x,y =xx.getWcs().skyToPixel(radec)  
        wwcs = xx.getWcs()
        # Define a small region for a cutout
        bbox1 = lsst.geom.Box2I(lsst.geom.Point2I(x-(ddim/2), y-(ddim/2)), lsst.geom.Extent2I(ddim,ddim))
        poly = makeSkyPolygonFromBBox(bbox, wwcs) 
        if poly.contains(point.getVector()):
            try:
                tmp  = xx.Factory(xx, bbox1, origin=afwImage.LOCAL, deep=False)
                cutout.append(tmp.image.array.flatten())
                cutout1.append(xx.visitInfo.getDate().get(lsst.daf.base.DateTime.MJD, lsst.daf.base.DateTime.UTC))
                cutout2.append(xx.visitInfo.getDate())     
            except:
                print("the cutout has exceeded the dimension of the image")
                
    # take date element
    from datetime import datetime
    def takeSecond(elem):
        #date = datetime.strptime(elem.visitInfo.getDate().strftime("%m/%d/%Y, %H:%M:%S"),'%Y-%m-%d %H:%M:%S')
        return elem.get(lsst.daf.base.DateTime.MJD, lsst.daf.base.DateTime.UTC)
    cutout2.sort(key= takeSecond)
    cut = zip(cutout1, cutout)
    try:
        #cut1 = sorted(cut)
        sort_cut = [element for _, element in sorted(cut)]

        ix= np.arange(ddim,dtype=float)
        iy= np.arange(ddim,dtype=float)
        pixPointList = [lsst.geom.Point2D(u, v) for u in ix for v in iy]
        
        dataId = {'visit': record_query[0].dataId['visit'], 'detector': record_query[0].dataId['detector']}
        xx= butler.get('calexp', dataId= registry.expandDataId(dataId))
        spherePoints = xx.getWcs().pixelToSky(pixPointList)
        #parentSkyPos =  cutout[0].getWcs().pixelToSkyArray(ix,iy,degrees=True)
        r_m=[u.getRa().asDegrees() for u in spherePoints]
        d_m=[u.getDec().asDegrees() for u in spherePoints]

        animation= []

        dim_grid=15
        X= np.zeros((dim_grid,dim_grid))
        Y= np.zeros((dim_grid,dim_grid))

        for i in range(dim_grid):
            for j in range(dim_grid):
                X[i,j]=spherePoints[(i*ddim+j)].getRa().asDegrees()
        for i in range(dim_grid):
            for j in range(dim_grid):
                Y[i,j]=spherePoints[(i*ddim+j)].getDec().asDegrees()

        for i in sort_cut:
            im = griddata((r_m,d_m),i,(X,Y),method='cubic',fill_value= np.mean(i))
            animation.append(im)
        
        #replace the path folder with local path folder
        #in the local path folder run mkir DATA
        #in the local path folder run mkir PLOT        
        !rm /home/antoniovanzanella/DATA/*.fits
        !rm /home/antoniovanzanella/PLOT/*.png

        for y in range(0,len(animation)):
            hdu = fits.PrimaryHDU(animation[y])
            hdu.header['DATE'] = str(cutout2[y])
            hdu.header[''] = str(rand_RA)
            hdu.header['COMMENT'] = str(rand_DEC)
            hdul = fits.HDUList([hdu])
        
            #replace the path folder with local path folder
        
            hdul.writeto("/home/avanzan/DATA/new%02d.fits" %y)
            fits_image="/home/avanzan/DATA/new00.fits"
            plt.title(cutout2[y])
            plt.imshow(animation[y], cmap='gray', vmin=-100.0, vmax=200, origin='lower')
            plt.savefig("/home/avanzan/PLOT/test%02d.png" %y, cmap='gray', vmin=-100.0, vmax=200, origin='lower')
            #plt.show()

        numbers = re.compile(r'(\d+)')
    
        def numericalSort(value):
            parts = numbers.split(value)
            parts[1::2] = map(int, parts[1::2])
            return parts

        def make_gif(frame_folder):
            frames = [Image.open(image) for image in sorted(glob.glob(f"{frame_folder}/*.png"),key= numericalSort)]
            frame_one = frames[0]
            frame_one.save("interp.gif", format="GIF", append_images=frames,
                       save_all=True, duration=700, loop=0)

        make_gif("/home/avanzan/PLOT/")  
        name = "samples_"+ str(g)+".tar.gz"
        !export name
        !tar -czf $name DATA
    except:
        print("EMPTY SEQUENCE")  

# Step by Step

## Preliminary operations
First of all, we imported all the libraries to query and process the LSST data.
Furthermore, other packages were used to interpolate and stack the samples and to handle files.

In [47]:
#to handle and process LSST data
import lsst.daf.butler as dafButler
import lsst.sphgeom
import astropy.time
import lsst.geom
import lsst.afw.display as afwDisplay
import lsst.afw.image as afwImage
from lsst.skymap import makeSkyPolygonFromBBox

#to interpolate the data
from scipy.interpolate import griddata 
from astropy.io import fits

#to handle files and display images
from random import uniform
import re
import matplotlib.pyplot as plt
import glob
from PIL import Image
from datetime import datetime
import numpy as np

Using butler to connect with the DP0.2 repository.

In [48]:
repo = 'dp02'  
collection='2.2i/runs/DP0.2'
butler = dafButler.Butler(repo,collections=collection)
registry = butler.registry

## The query

We pick a random coordinates pair in the LSST simulated sky and we chose a correct pixelization around that point.
We use a specific level(12) of the HTM (hierarchical triangular mesh) pixelization of the sky ([HTM primer](http://www.skyserver.org/htm/)).
<br>"The process is to transform a region or point into one or more HTM identifiers (HTM IDs), and then create a query using the HTM ID as the spatial data ID.
The `lsst.sphgeom` library supports region objects and HTM pixelization in the LSST Science Pipelines."<br>(Taken from the **04_Intro_to_Butler** notebook).

In [49]:
RA_lower_limit = 50
RA_upper_limit = 74
DEC_lower_limit = -44
DEC_upper_limit = -27
rand_RA = uniform(RA_lower_limit,RA_upper_limit)
rand_DEC =uniform(DEC_lower_limit,DEC_upper_limit) 
pixelization = lsst.sphgeom.HtmPixelization(12)
htm_id = pixelization.index(
    lsst.sphgeom.UnitVector3d(
        lsst.sphgeom.LonLat.fromDegrees(rand_RA, rand_DEC)   
    )
)

#scale = pixelization.triangle(htm_id).getBoundingCircle().getOpeningAngle().asDegrees()*3600
#print(f'HTM ID={htm_id} at level={pixelization.getLevel()} is a ~{scale:0.2}" triangle.')

Now, we set a reference time.
Then we make a query to obtain all the calexp images observed during the chosen time period(e.g. roughly 4 years).

In [50]:
t = astropy.time.core.Time("2025-09-05 08:16:03.643200")

minute = astropy.time.TimeDelta(60, format="sec")
timespan = dafButler.Timespan(t - minute*1051200,t + minute*1051200)#4 years
#timespan = dafButler.Timespan(t - minute*2600000,t + minute*2600000)#10 years
#timespan = dafButler.Timespan(t - minute*525600,t + minute*525600)#1 years

datasetRefs1 = registry.queryDatasets("calexp", htm20=htm_id,
                                         where="visit.timespan OVERLAPS my_timespan",
                                         bind={"my_timespan": timespan})

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-5.1.0/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


All the  retrieved images are saved in a list.

In [51]:
record_query= []  
for i, ref in enumerate(datasetRefs1):
    record_query.append(ref)

In [52]:
print(len(record_query))

540


## Manage data

These lists will contain the flattened images, the Julian dates, and the UTC respectively.

In [53]:
cutout = []
cutout_date_MJD = []
cutout_date = []

point = lsst.geom.SpherePoint(rand_RA,rand_DEC, lsst.geom.degrees)

For each calexp "record" we asked for the respective image and its metadata; using lsst.geom we defined an interest region,
<br> a squared bounding box mapped on the sky all around the chosen point.
To be able to work with this information we converted the coords into pixels and define a box to delimit the portion of the image we want to extract(the cut-out).
<br>Then we append to the list the image and the linked information only if the area is contained in the current image. Otherwise, an exception is generated.

In [54]:
#ddim= 100
ddim= 50
#It requires too much time.
#for i in range(0,len(record_query)):
for i in range(0,50):
    dataId = {'visit': record_query[i].dataId['visit'], 'detector': record_query[i].dataId['detector']}
    xx= butler.get('calexp', dataId= registry.expandDataId(dataId))
    radec= lsst.geom.SpherePoint(rand_RA*lsst.geom.degrees, rand_DEC*lsst.geom.degrees)
    x,y =xx.getWcs().skyToPixel(radec)  
    wwcs = xx.getWcs()
    # Define a small region for a cutout
    bbox1 = lsst.geom.Box2I(lsst.geom.Point2I(x-(ddim/2), y-(ddim/2)), lsst.geom.Extent2I(ddim,ddim))
    poly = makeSkyPolygonFromBBox(bbox1, wwcs) 
    if poly.contains(point.getVector()):
        try:
            tmp  = xx.Factory(xx, bbox1, origin=afwImage.LOCAL, deep=False)
            cutout.append(tmp.image.array.flatten())
            cutout_date_MJD.append(xx.visitInfo.getDate().get(lsst.daf.base.DateTime.MJD, lsst.daf.base.DateTime.UTC))
            cutout_date.append(xx.visitInfo.getDate())     
        except:
            print("the cutout has exceeded the dimension of the image")

the cutout has exceeded the dimension of the image
the cutout has exceeded the dimension of the image
the cutout has exceeded the dimension of the image
the cutout has exceeded the dimension of the image
the cutout has exceeded the dimension of the image
the cutout has exceeded the dimension of the image
the cutout has exceeded the dimension of the image
the cutout has exceeded the dimension of the image
the cutout has exceeded the dimension of the image
the cutout has exceeded the dimension of the image
the cutout has exceeded the dimension of the image
the cutout has exceeded the dimension of the image
the cutout has exceeded the dimension of the image
the cutout has exceeded the dimension of the image
the cutout has exceeded the dimension of the image
the cutout has exceeded the dimension of the image
the cutout has exceeded the dimension of the image
the cutout has exceeded the dimension of the image
the cutout has exceeded the dimension of the image
the cutout has exceeded the dim

In [55]:
print(len(cutout))

27


Here we implemented a function that returns the value in the data field.<br>
Subsequently, this will be used to sort the list by time.

In [56]:
def takeSecond(elem):
    return elem.get(lsst.daf.base.DateTime.MJD, lsst.daf.base.DateTime.UTC)

In [57]:
cutout_date.sort(key= takeSecond)
cut = zip(cutout_date_MJD, cutout)

cut1 = sorted(cut)
sort_cut = [element for _, element in cut1]

## Interpolation

Then we create a mask of points to interpolate the images choosing a random image to initialize the procedure.
<br> Using lstm.geom we generate two arrays of points that are converted in RA and DEC coordinates according to the current image properties.
Defining two small grids for RA and DEC in degrees, we apply a bicubic interpolation (griddata, https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.griddata.html )<br> to our cut-outs.
We made this since the different times when the images are observed produced a sequence of images in which the sky rotated and translated.

In [58]:
ix= np.arange(ddim,dtype=float)
iy= np.arange(ddim,dtype=float)

pixPointList = [lsst.geom.Point2D(u, v) for u in ix for v in iy]

#we choose a random image to embed our grid.
dataId = {'visit': record_query[0].dataId['visit'], 'detector': record_query[0].dataId['detector']}
xx= butler.get('calexp', dataId= registry.expandDataId(dataId))
spherePoints = xx.getWcs().pixelToSky(pixPointList)
#the reverse operation 
#parentSkyPos =  cutout[0].getWcs().pixelToSkyArray(ix,iy,degrees=True)

r_m=[u.getRa().asDegrees() for u in spherePoints]
d_m=[u.getDec().asDegrees() for u in spherePoints]

The interpolation is a lossy function and this is the reason why we took a much larger area than the cut-outs from the images in the previous code fragments.
<br> The final cut-outs are 15 by 15 pixels.

In [59]:
animation= []

dim_grid=15
X= np.zeros((dim_grid,dim_grid))
Y= np.zeros((dim_grid,dim_grid))

for i in range(dim_grid):
    for j in range(dim_grid):
        X[i,j]=spherePoints[(i*ddim+j)].getRa().asDegrees()
for i in range(dim_grid):
    for j in range(dim_grid):
        Y[i,j]=spherePoints[(i*ddim+j)].getDec().asDegrees()

for i in sort_cut:
    im = griddata((r_m,d_m),i,(X,Y),method='cubic',fill_value= np.mean(i))
    animation.append(im)

In [ ]:
print(len(animation))

## Data storage 

All the images produced are stacked to build a single sample. By finding ourselves in a cycle we need to remove old images and this explains the presence of the rm command at the beginning of the section.<br> Later, for each image in the sequence we save it in a **fits** file using the `astropy.io` library.
The header is used to save time and coordinates information.

In [ ]:
#
#replace the path folder with local path folder
#
!rm /home/avanzan/DATA/*.fits
!rm /home/avanzan/PLOT/*.png

for y in range(0,len(animation)):
    hdu = fits.PrimaryHDU(animation[y])
    hdu.header['DATE'] = str(cutout_date[y])
    hdu.header[''] = str(rand_RA)
    hdu.header['COMMENT'] = str(rand_DEC)
    #hdu.header['RA'] = str(rand_RA)
    #dhu.header['DEC']= str(rand_DEC)
    hdul = fits.HDUList([hdu])
    hdul.writeto("/home/avanzan/DATA/new%02d.fits" %y)
    fits_image="/home/avanzan/DATA/new00.fits"
    plt.title(cutout_date[y])
    plt.imshow(animation[y], cmap='gray', vmin=-100.0, vmax=200, origin='lower')
    plt.savefig("/home/avanzan/PLOT/test%02d.png" %y)
                #, cmap='gray', vmin=-100.0, vmax=200, origin='lower')

numbers = re.compile(r'(\d+)') 

We have two extra functions implemented to make a gif of the sample.<br> These, are not strictly necessary but we built them to have a visual output of our work.
Finally, the sample is compressed using **tar** algorithm.

In [64]:
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

def make_gif(frame_folder):
    frames = [Image.open(image) for image in sorted(glob.glob(f"{frame_folder}/*.png"),key= numericalSort)]
    frame_one = frames[0]
    frame_one.save("interp.gif", format="GIF", append_images=frames,save_all=True, duration=700, loop=0)

make_gif("/home/avanzan/PLOT")  
### g as input works only in the for loop
#name = "samples_"+ str(g)+".tar.gz"
name = "samples_"+ str(0)+".tar.gz"

!export name
!tar -czf $name DATA 

Iterating this procedure, we built a great amount of samples where we injected simulated Slow-moving objects starting from TNOs ephemeris taken from the JPL database.

## Slow moving objects injection

<div>
<img src="workflow.png" width="500"/>        
</div>

Unfortunately, the remaining part of the Dataset creation requires to move on a different hardware and this is the reason behind the file storage.
As is possible to see in the upper figure, we used these samples as input in another function.
<br> We use one of the **JPL** tools to query about all the: TNOs, Parabolic and Hyperbolic Asteroids, and Hyperbolic and Parabolic Comets present in the dataset. 
<br>Once obtained the objects and relative information, such as absolute magnitude, eccentricity, perihelion distance, semi-major axis, inclination, and so on, we store them in a **CSV** file. 
<br>Starting from this information we use the **astropy** library to compute the ephemerides of the objects every 24 hours for five years, taking care to overlap the range of time of computation with the one used for the query on the simulated LSST image;
<br> then we save all the RA and DEC values in arrays as attributes of a python class together with the other object's information. 
<br>In this way, at the end we have a series of instances of a class, where each instance is a different celestial object with its ephemerides and physical characteristics.

Object injection into the images is first done by creating a 15x15 grid centered on zero and checking that each object falls during the sequence in the cutouts. 
<br>For this purpose we check the standard deviation of the grid and we make sure that its standard deviation is greater than the standard deviation of the arrays of right ascension and declination. 
<br>Then we paint the object on the grid using a PSF (Point Spread Function). We do this for each frame, by changing the position of the PSF on the basis of the ephemerides we downloaded from JPL.
<br>However, since the objects we took from the JPL database are too close to us than what we would like to detect, we project them at a greater distance by changing a scaling factor during the injection.


<div>
<img src="intime.png" width="500"/>        
</div>


The last step of the dataset population process is to overlap the grid containing the slow-moving object to the LSST image.
<br>It is important to note that we want the movement described in the grids maintain the conceptual consistency of the LSST images;
<br> this is the reason We made a function that perfectly matches the given observation date of the LSST images to one of the object ephemerides. 
<br>When there is no match, the function simply discards the grid from the sequence and goes on.


<div>
<img src="sim.png" width="500"/>        
</div>

## 3D-CNN

Beyond the classic structure that provides input-convolutional-fully connected layers, we applied 3D max-pooling to avoid the uncontrolled features map change of dimension;
<br>dropout is strongly recommended to avoid the overfitting problem.
<br>In fact, those data showed a great tendency to this phenomenon. Probably, It is caused by their similarity. 
<br>We also used batch normalization to avoid the shift covariance phenomenon and the L2-regularization.
<br>After several of these blocks-like a dense(fully-connected) layer has been placed and after it, a single neuron able to make the binary classification. 
<br>ReLu was used as the activation function and the Binary cross-entropy as loss function.


<div>
<img src="CNN (2).png" width="700"/>        
</div>


The NN takes video as patterns, in particular a 50 frames series of 15 by 15 size forwarding them through the Convolutional blocks. 
<br>Each of the six convolutional layers increases the depth of the feature maps, starting from 16 filters in the first convolution to 512 in the last one. 
<br>After each convolution a batch normalization and a 3D max pooling are used. 
<br>Finally, a fully connected layer working on the flattened features maps is used in anticipation of the Sigmoid activation function in charge of providing output. As loss function a binary cross-entropy is used and a ReLu as activation function. 
<br>As output the network discriminates against sequences with the presence of the slow-moving object or not.